**Import**

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns  
from matplotlib.pyplot import figure
import string
import os
import tensorflow as tf
import cv2

In [ ]:
data=pd.read_csv('/content/movie.csv')
data.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [ ]:
max_length = 0
for sentence in data["text"]:
    new_sentence = sentence.translate(str.maketrans("","",string.punctuation))
    #print(new_sentence)
    length = len(new_sentence.split(" "))
    if length > max_length:
        max_length = length

In [ ]:
data["label"].value_counts()

0    20019
1    19981
Name: label, dtype: int64

**With this function we will assign each word in the dataset, an integer. This function returns a dictionary**

In [ ]:
def tokenize_the_data_from_pandas(dataframe,column_name): # This function returns tokens dictionary that includes every unique word as keys and unique integers for each key.
    #Here we clean the string out of punctuations.
    import string
    for sent in range(len(data[column_name])):
        example_sentence = data[column_name].iloc[sent]
        new_sentence = example_sentence.translate(str.maketrans("","",string.punctuation))
        data[column_name].iloc[sent] = new_sentence
    #Here we create a dictionary that will encode each word into an integer to have a representation of the word in the deep neural networks processes
    tokens = {}
    for sent in range(len(data[column_name])):
        example_sentence = data[column_name].iloc[sent]
        values = example_sentence.split(" ")
        for word in values:
            tokens[word] = 0
         
    names = list(tokens.keys())
    for num in range(len(names)):
        tokens[names[num]] = num+1
        
    return tokens

In [ ]:
tokens = tokenize_the_data_from_pandas(data,column_name="text")

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
len(tokens.keys())

190010

**Creating the data vectors**

In [ ]:
X = []
y = []
for sent in range(len(data["text"])):
    example_sentence = data["text"].iloc[sent]
    values = example_sentence.split(" ")
    for word in range(len(values)):
        values[word] = tokens[values[word]]
    X.append(values)
    y.append(data["label"].iloc[sent])
    

#This code will pad the sequences with zeros to make every input the same length.
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = max_length
padded_docs = pad_sequences(X, maxlen=max_length, padding='post')
#the code below changes the final data lists into numpy arrays so that we can train our neural network with.    
X_ar = np.array(padded_docs)
y_ar = np.array(y)
#Here we split our data into train and test sets.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_ar,y_ar,test_size=0.1,random_state = 10)

**This is a 1D convolutional model for the task**

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(190020,200,input_length=2470))#This embedding layer reduces the data dimension from 290 to 10 by creating relations and using floating numbers to represent the words.
model.add(tf.keras.layers.Conv1D(32,7,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(64,7,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(5))

model.add(tf.keras.layers.Conv1D(64,7,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(5))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = "adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
1125/1125 [==============================] - 1299s 1s/step - loss: 0.3613 - accuracy: 0.8152 - val_loss: 0.2119 - val_accuracy: 0.9165
Epoch 2/5
1125/1125 [==============================] - 1294s 1s/step - loss: 0.1044 - accuracy: 0.9638 - val_loss: 0.2426 - val_accuracy: 0.9090
Epoch 3/5
1125/1125 [==============================] - 1290s 1s/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.5594 - val_accuracy: 0.8840
Epoch 4/5
1125/1125 [==============================] - 1291s 1s/step - loss: 0.0067 - accuracy: 0.9981 - val_loss: 0.4611 - val_accuracy: 0.9022
Epoch 5/5
1125/1125 [==============================] - 1293s 1s/step - loss: 0.0109 - accuracy: 0.9963 - val_loss: 0.5609 - val_accuracy: 0.8882


**Predicting the test set to investiagte further**

In [ ]:
y_pred=model.predict(X_test[:])

**We turn our networks output into binary values**

In [ ]:
y_pred[y_pred>0.5] = 1
y_pred[y_pred< 0.5] = 0

In [ ]:
Truth = 0
Falset = 0
for pik in range(len(y_pred)):
    
    result = y_pred[pik] == y_test[pik]
    if False in result:
        Falset += 1 
    else:
        Truth += 1

In [ ]:
print(Truth,Falset)

3553 447


In [ ]:
print("accuracy:",Truth / (Truth + Falset))

accuracy: 0.88825
